In [1]:
# pip install -U langgraph langchain-google-genai langchain-core python-dotenv google-generativeai

In [1]:
import os
import sys
from typing import TypedDict, Optional
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

try:
    API_KEY = os.environ["GEMINI_PAID_KEY"]
except KeyError:
    print("Error: The 'GEMINI_PAID_KEY' environment variable is not set in your .env file.")
    sys.exit(1)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",  
    google_api_key=API_KEY,
    temperature=0.7,
    convert_system_message_to_human=True
)

class HackathonState(TypedDict):
    idea: str
    brainstorm: Optional[str]
    design: Optional[str]
    code: Optional[str]
    test: Optional[str]
    presentation: Optional[str]

# Define System Prompts 
PROMPTS = {
    "brainstorm": """You are a hackathon brainstorming assistant. 
    Input: {idea}.
    Output JSON: {{ "title": "...", "summary": "...", "features": [], "milestones": [], "tech_stack": [] }}.
    Be concise.""",
    
    "design": """You are a UX designer. 
    Input Idea: {idea}
    Brainstorm Data: {brainstorm}
    Output JSON: {{ "screens": [], "wireframes": [], "ui_notes": "..." }}. 
    Be concise.""",
    
    "code": """You are a developer coach.
    Input Idea: {idea}
    Design Specs: {design}
    Output JSON: {{ "snippets": [{{ "name": "...", "language": "...", "code": "...", "explanation": "..." }}], "setup_steps": [] }}.""",
    
    "test": """You are a QA advisor.
    Input Code Plan: {code}
    Output JSON: {{ "checklist": [], "example_unit_tests": [] }}.""",
    
    "presentation": """You are a presentation coach.
    Full Project Context: 
    - Idea: {idea}
    - Brainstorm: {brainstorm}
    - Design: {design}
    Output: A 3-minute presentation script and slide cues in JSON: {{ "script": "...", "slide_cues": [] }}."""
}

# Each function receives the current state, runs the LLM, and returns the *diff* to update the state.

def brainstorm_step(state: HackathonState):
    print("--- Brainstorming ---")
    msg = PROMPTS["brainstorm"].format(idea=state["idea"])
    response = llm.invoke(msg)
    return {"brainstorm": response.content}

def design_step(state: HackathonState):
    print("--- Designing ---")
    # We can access previous outputs (brainstorm) from the state
    msg = PROMPTS["design"].format(idea=state["idea"], brainstorm=state["brainstorm"])
    response = llm.invoke(msg)
    return {"design": response.content}

def code_step(state: HackathonState):
    print("--- Coding ---")
    msg = PROMPTS["code"].format(idea=state["idea"], design=state["design"])
    response = llm.invoke(msg)
    return {"code": response.content}

def test_step(state: HackathonState):
    print("--- Testing ---")
    msg = PROMPTS["test"].format(code=state["code"])
    response = llm.invoke(msg)
    return {"test": response.content}

# Graph creation
workflow = StateGraph(HackathonState)

# Add Nodes
workflow.add_node("brainstorm", brainstorm_step)
workflow.add_node("design", design_step)
workflow.add_node("code", code_step)
workflow.add_node("test", test_step)

# Define Edges 
workflow.set_entry_point("brainstorm")
workflow.add_edge("brainstorm", "design")
workflow.add_edge("design", "code")
workflow.add_edge("code", "test")
workflow.add_edge("test", END)

# Compile the graph
app = workflow.compile()

#  On-Demand Presentation if the user click y ---
def generate_presentation(final_state: HackathonState):
    """
    This runs separately from the main graph, using the final accumulated state.
    """
    print("\nGenerating Presentation Script...")
    msg = PROMPTS["presentation"].format(
        idea=final_state["idea"],
        brainstorm=final_state["brainstorm"],
        design=final_state["design"]
    )
    response = llm.invoke(msg)
    return response.content

if __name__ == "__main__":
    user_idea = "emotion detection using CNN"
    
    print(f"Starting Hackathon Pipeline for: '{user_idea}'\n")
    initial_state = {"idea": user_idea}
    
    # Run the pipeline (Brainstorm -> Design -> Code -> Test)
    final_state = app.invoke(initial_state)
    
    print("\nPipeline Complete!")
    print("-" * 40)
    print("Sample Output (Tech Stack from Brainstorming):")
    # Just printing a snippet to verify it worked
    print(final_state["brainstorm"][:200] + "...") 
    print("-" * 40)

    user_input = input("\nWould you like to generate the presentation script? (y/n): ")
    if user_input.lower().startswith('y'):
        script = generate_presentation(final_state)
        print("\nPresentation Script:")
        print(script)

Starting Hackathon Pipeline for: 'emotion detection using CNN'

--- Brainstorming ---
--- Designing ---
--- Coding ---
--- Testing ---

Pipeline Complete!
----------------------------------------
Sample Output (Tech Stack from Brainstorming):
```json
{
  "title": "EmoNet: Real-Time Emotion Recognition",
  "summary": "A web application that uses a Convolutional Neural Network (CNN) to detect and classify human emotions in real-time from a w...
----------------------------------------

Generating Presentation Script...

Presentation Script:
```json
{
  "script": "(Start on Title Slide)\n\nHello everyone. Think about our digital conversations—on video calls, in chat—they often lack a critical piece of human connection: emotional context. We see faces, but we miss the subtle cues. How can we build technology that’s more empathetic and aware? \n\n(Click to next slide)\n\nToday, I'm introducing EmoNet, a project designed to bridge that emotional gap. EmoNet is a real-time emotion recognition